# Day 13: Problem 2
This problem was new to me. I needed to learn a little bit about number thoery.

## The problem
Let the index of the bus be given by its (zero-indexed) location in the list. For example, with a list "17,x,13,19", Bus 0 is $b_0=17$, Bus 2 is $b_2 = 13$ and Bus 3 is $b_3 = 19$. We want to find a timestamp $t$, such that Bus 1 leaves at that timestamp, and all other buses leave at timestamp $t + i$, where $i$ is the index of the bus. In this example, Bus 1 would leave at time $t$; Bus 2 would leave at time $t+2$; and Bus 3 would leave at time $t+3$. 

This timestamp $t$ is the solution to the following system of equations:
$$ t+i~ (\text{mod}~b_i) = 0, \forall i~\text{in the list of buses}$$

Rearranging this becomes, 
$$ t = -i~ (\text{mod}~b_i), \forall i~\text{in the list of buses}$$

**[The Chinese Remainder Theorem](https://crypto.stanford.edu/pbc/notes/numbertheory/crt.html)** Gives a method to solve a system of equations given by the latter form.

For our example, this is written as:
$$
t = 0~(\text{mod}~ 17) \\
t = -2~(\text{mod}~ 13) \\
t = -3~(\text{mod}~ 19)
$$

## Chinese Remainder Theorem
Let $M = \prod_i b_i$. Define $q_i = M/b_i$ and $q'_i = q_i^{-1}~(\text{mod}~b_i)$. Then the solution is given by

$$ t = \sum_i -i q_i q'_i~(\text{mod}~M).$$

The only thing missing is to determine the inverse $b'_i$. For this inverse to exist the greatest common denominator must satisfy $\text{gcd}(q_i,b_i) = 1$; *i.e.,* they are coprime. The proof of existence of such an inverse and the inverse itself can be computed using ([Euclid's algorithm](https://crypto.stanford.edu/pbc/notes/numbertheory/euclid.html) is one method to show existence (gcd =1) as well as [compute the inverse](https://crypto.stanford.edu/pbc/notes/numbertheory/division.html). 

*Note: Julia has a built in function to compute this inverse:* ```invmod```

*Note 2: The ```Mods``` Julia package has a function that computes the Chinese Remainder Theorem:* ```CRT```

In [54]:
function getBuses(input)
    if typeof(input) == String # Need this to deal with my test cases as well as reading in the input
        buses = split(input,',')
    else 
        buses = split(input[2],',')
    end
    ans = Array{Int128,1}()
    ans2 = Array{Int128,1}()
    count = 0
    for b in buses
        if b != "x" 
            push!(ans,parse(Int64,b))
            push!(ans2,count)
        end
        count += 1
    end
    return ans,ans2
end

getBuses (generic function with 1 method)

In [90]:
function bezout(a,b)
#     if m > b
#          rkm1 = m
#          rk = b
#     else
#         rkm1 = b
#         rk = m
#     end
    rkm1 = a
    rk = b
    skm1,tkm1,sk,tk = 1,0,0,1
    rkp1 = 1 # Initialize to start while loop
    while rkp1 != 0
        q = rkm1 ÷ rk 
        rkp1 = rkm1 - q*rk # Get remainder
#         println("Quotient: ",q," r{k-1} = ",rkm1," r{k} = ",rk," r{k+1} = ",rkp1)
        rkm1 = rk # Update values for next iteration
        rk = rkp1
#         println("r{k-1} = ",rkm1," r{k} = ",rk," r{k+1} = ",rkp1)
        skp1 = skm1 - q*sk
        tkp1 = tkm1 - q*tk
        skm1 = sk
        tkm1 = tk
        tk = tkp1
        sk = skp1
    end
    return rkm1,skm1,tkm1 # GCD, Bezout1, Bezout 2
end
function inverse(b,m)
    # Julia has a built in function to do this <invmod>
    answer = bezout(b,m)
    if answer[1] == 1
        return mod(answer[2],m)
    end
end
function my_CRT(ind,input)
    M = prod(input)
    return mod(sum(-ai * inverse(M ÷ ni, ni) * M ÷ ni for (ni,ai) in zip(input,ind)),M)
end

my_CRT (generic function with 1 method)

### Calculates the value of the chinese remainder theorem
$$ x = \sum_i i q_i q'_i~(\text{mod}~M).$$

**Note:** By inspection, the bus numbers are all coprime (this holds for the given input for this problem). This is helpful and eliminates a step needed to find the greatest common denominator of all the buses. 

In [95]:
using Mods
# Check the solution using Mods Chinese Remainder Theorem
println("Test 1")
test1 = "17,x,13,19" # 3417
input,ind = getBuses(test1)
@show my_CRT(ind,input)
@show CRT(Mod{17}(0),Mod{13}(-2),Mod{19}(-3))
println("\nTest 2")
test1 = readlines("../testdata/day_13.txt") # 1068781
input,ind = getBuses(test1)
@show my_CRT(ind,input)
@show CRT(Mod{7}(0),Mod{13}(-1),Mod{59}(-4),Mod{31}(-6),Mod{19}(-7))

Test 1
my_CRT(ind, input) = 3417
CRT(Mod{17}(0), Mod{13}(-2), Mod{19}(-3)) = Mod{4199}(3417)

Test 2
my_CRT(ind, input) = 1068781
CRT(Mod{7}(0), Mod{13}(-1), Mod{59}(-4), Mod{31}(-6), Mod{19}(-7)) = Mod{3162341}(1068781)


Mod{3162341}(1068781)

# The solution to today's problem

In [98]:
test1 = readlines("../data/day_13.txt") 

input,ind = getBuses(test1)
println("The solution is: ",my_CRT(ind,input))


The solution is: 894954360381385
